## High-dimensional dynamics of generalization error in neural networks

Attempt to reproduce Figure 5B in the paper.


In [ ]:
#!/usr/bin/env python
# coding: utf-8

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import argparse
import os
import datetime
import pathlib
import random
import json
import numpy as np
import math

import torch

import sys
sys.path.append('../code/')
from linear_utils import linear_model
from train_utils import save_config

In [ ]:
# argument written in command line format
cli_args = '--seed 12 --save-results --jacobian --risk-loss L2 -t 50000 -w 0.01 0.01 --lr 0.000001 -d 50 -n 100 --hidden 50 --sigmas 1'
sigma_noise = 0.0


In [ ]:
"""
A fully-connected ReLU network with one hidden layer, trained to predict y from x
by minimizing the MSE loss.
"""

# get CLI parameters
parser = argparse.ArgumentParser(description='CLI parameters for training')
parser.add_argument('--root', type=str, default='', metavar='DIR',
                    help='Root directory')
parser.add_argument('-t', '--iterations', type=int, default=1e4, metavar='ITERATIONS',
                    help='Iterations (default: 1e4)')
parser.add_argument('-n', '--samples', type=int, default=100, metavar='N',
                    help='Number of samples (default: 100)')
parser.add_argument('--print-freq', type=int, default=100,
                    help='CLI output printing frequency (default: 1000)')
parser.add_argument('--gpu', type=int, default=None,
                    help='Number of GPUS to use')
parser.add_argument('--seed', type=int, default=None,
                    help='Random seed')                        
parser.add_argument('-d', '--dim', type=int, default=50, metavar='DIMENSION',
                    help='Feature dimension (default: 50)')
parser.add_argument('--hidden', type=int, default=200, metavar='DIMENSION',
                    help='Hidden layer dimension (default: 200)')
parser.add_argument('--sigmas', type=str, default=None,
                    help='Sigmas')     
parser.add_argument('-r','--s-range', nargs='*', type=float,
                    help='Range for sigmas')
parser.add_argument('-w','--scales', nargs='*', type=float,
                    help='scale of the weights')
parser.add_argument('--lr', type=float, default=1e-4, nargs='*', metavar='LR',
                    help='learning rate (default: 1e-4)')              
parser.add_argument('--normalized', action='store_true', default=False,
                    help='normalize sample norm across features')
parser.add_argument('--risk-loss', type=str, default='MSE', metavar='LOSS',
                    help='Loss for validation')
parser.add_argument('--jacobian', action='store_true', default=False,
                    help='compute the SVD of the jacobian of the network')
parser.add_argument('--save-results', action='store_true', default=False,
                    help='Save the results for plots')
parser.add_argument('--details', type=str, metavar='N',
                    default='no_detail_given',
                    help='details about the experimental setup')


args = parser.parse_args(cli_args.split())

# directories
root = pathlib.Path(args.root) if args.root else pathlib.Path.cwd().parent

current_date = str(datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S'))
args.outpath = (pathlib.Path.cwd().parent / 'results' / 'two_layer_nn' /  current_date)

if args.save_results:
    args.outpath.mkdir(exist_ok=True, parents=True)

if args.seed is not None:
    random.seed(args.seed)
    torch.manual_seed(args.seed)
    np.random.seed(args.seed)
    
device = torch.device('cpu')
# device = torch.device('cuda') # Uncomment this to run on GPU

In [ ]:
d_out = 1      # dimension of y

# sample training set from the linear model
lin_model = linear_model(args.dim, sigma_noise=sigma_noise, normalized=False, sigmas=args.sigmas, s_range=args.s_range)
Xs, ys = lin_model.sample(args.samples)
Xs = torch.Tensor(Xs).to(device)
ys = torch.Tensor(ys.reshape((-1,1))).to(device)

# sample the set for empirical risk calculation
Xt, yt = lin_model.sample(args.samples)
Xt = torch.Tensor(Xt).to(device)
yt = torch.Tensor(yt.reshape((-1,1))).to(device)

In [ ]:
# define loss functions
loss_fn = torch.nn.MSELoss(reduction='sum')
risk_fn = torch.nn.L1Loss(reduction='mean') if args.risk_loss == 'L1' else loss_fn

### Empirical

In [ ]:
model = torch.nn.Sequential(
           torch.nn.Linear(args.dim, args.hidden),
           torch.nn.Linear(args.hidden, d_out),
         ).to(device)      
                
# use kaiming initialization                
if args.scales:
    i = 0
    with torch.no_grad():
        for m in model:
            if type(m) == torch.nn.Linear:
                if i == 0:
                    torch.nn.init.kaiming_normal_(m.weight, a=math.sqrt(5))
                    m.weight.data = torch.mul(m.weight.data, args.scales[0])
                if i == 1:
                    torch.nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))
                    m.weight.data = torch.mul(m.weight.data, args.scales[1])
                i += 1
                

# use same learning rate for the two layers
if isinstance(args.lr, list):
    stepsize = [max(args.lr)] * 2

In [ ]:
# train the network
losses_same = []
risks_same = []
for t in range(int(args.iterations)):
    y_pred = model(Xs)

    loss = loss_fn(y_pred, ys)
    losses_same.append(loss.item())

    if not t % args.print_freq:
        print(t, loss.item())

    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        i = 0
        for param in model.parameters():
            param.data -= stepsize[i] * param.grad
            if not len(param.shape) > 1:
                i += 1
            
    with torch.no_grad():
        yt_pred = model(Xt)
        
        risk = risk_fn(yt_pred, yt)
        risks_same.append(risk.item())

        if not t % args.print_freq:
            print(t, risk.item())

In [ ]:
geo_samples = [int(i) for i in np.geomspace(1, len(risks)-1, num=700)]

In [ ]:
risks = np.array(risks_same)
losses = np.array(losses_same)

cmap = matplotlib.cm.get_cmap('viridis')
colorList = [cmap(50/1000), cmap(350/1000)]
labelList = ['same stepsize', 'scaled stepsize']

fig = plt.figure(figsize=(12,8))
ax = plt.subplot(111)
ax.set_xscale('log')

ax.plot(geo_samples, risks[geo_samples], 
        color=colorList[1], 
        label=labelList[1],
        lw=4)

ax.legend(loc=1, bbox_to_anchor=(1, 1), fontsize='x-large',
    frameon=False, fancybox=True, shadow=True, ncol=1)
ax.set_ylabel('risk')
ax.set_xlabel(r'$t$ iterations')

ax2 = fig.add_subplot(211)
ax2.set_xscale('log')
ax2.plot(geo_samples, losses[geo_samples], 
        color=colorList[1], 
        label=labelList[1],
        lw=4)

ax2.set_ylabel('loss')
ax2.set_xlabel(r'$t$ iterations')

plt.show()

In [ ]:
# For the sake of not messing anything up
Xs_t, ys_t, Xt_t, yt_t = Xs.T, ys.T, Xt.T, yt.T

V, s, _ = np.linalg.svd(Xs_t, full_matrices=True)
S = torch.tensor(np.concatenate((s, np.zeros(args.dim - s.shape[0]))).reshape(1, -1), dtype=torch.float32)
St = ys_t @ Xs_t.T @ torch.tensor(V)

In [ ]:
def dt(u, z, S, St):
    assert S.shape == z.shape

    return (St - u * z * S)

def dzdt(u, z, S, St):
    
    return u * dt(u, z, S, St)


def dudt(u, z, S, St):
    assert S.shape == z.shape
    
    return (dt(u, z, S, St) @ z.T).squeeze()
    

In [ ]:
# Simulation
w_init = 0.01
u = torch.normal(0, torch.tensor(w_init))
z = torch.normal(0, torch.tensor(w_init), size=(1, args.dim))

losses_teo = []
risks_teo = []
for t in range(int(args.iterations)):
    
    #u = u - args.lr[0] * dudt(u, z, S, St)
    z = z - args.lr[0] * dzdt(u, z, S, St)
    
    
    Wtot = u * z @ V.T

    y_pred = Wtot @ Xs_t

    loss = loss_fn(y_pred.T, ys_t.T)
    losses_teo.append(loss.item())

    if not t % args.print_freq:
        print(t, loss.item())
        
    yt_pred = Wtot @ Xt_t

    risk = risk_fn(yt_pred.T, yt_t.T)
    risks_teo.append(risk.item())

    if not t % args.print_freq:
        print(t, risk.item())

In [ ]:
risks = np.array(risks_teo)
losses = np.array(losses_teo)

cmap = matplotlib.cm.get_cmap('viridis')
colorList = [cmap(50/1000), cmap(350/1000)]
labelList = ['same stepsize', 'scaled stepsize']

fig = plt.figure(figsize=(12,8))
ax = plt.subplot(111)
ax.set_xscale('log')

ax.plot(geo_samples, risks[geo_samples], 
        color=colorList[1], 
        label=labelList[1],
        lw=4)

ax.legend(loc=1, bbox_to_anchor=(1, 1), fontsize='x-large',
    frameon=False, fancybox=True, shadow=True, ncol=1)
ax.set_ylabel('risk')
ax.set_xlabel(r'$t$ iterations')

ax2 = fig.add_subplot(211)
ax2.set_xscale('log')
ax2.plot(geo_samples, losses[geo_samples], 
        color=colorList[1], 
        label=labelList[1],
        lw=4)

ax2.set_ylabel('loss')
ax2.set_xlabel(r'$t$ iterations')

plt.show()